In [2]:
import uproot
import awkward as ak
import numpy as np
import sklearn.metrics as m
import boost_histogram as bh
import glob
import os
from scipy.interpolate import interp1d
import boost_histogram as bh

from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec

from cycler import cycler
# import mplhep as hep
# plt.style.use(hep.style.ROOT)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

def _p4_from_ptetaphie(pt, eta, phi, energy):
    import vector
    vector.register_awkward()
    return vector.zip({'pt': pt, 'eta': eta, 'phi': phi, 'energy': energy})
def _p4_from_ptetaphim(pt, eta, phi, mass):
    import vector
    vector.register_awkward()
    return vector.zip({'pt': pt, 'eta': eta, 'phi': phi, 'mass': mass})

from concurrent.futures import ThreadPoolExecutor
from functools import reduce
from operator import add

## Boosted

In [3]:
# Categories
from types import SimpleNamespace
config = SimpleNamespace(
    categories = { # category name: (sample list, selection, label, color)
        "bkg_wjets": (["wjets"], "d.lep_pt > 0", r"Bkg: W+jets", "lightcyan"),
        "bkg_t_matched": (["ttbarsl", "wwsl", "twsl"], "~d.is_wcb & d.is_t_matched", r"Bkg ($t_{bqq'}$)", "blue"),
        "bkg_w_matched": (["ttbarsl", "wwsl", "twsl"], "~d.is_wcb & d.is_w_matched", r"Bkg ($W_{qq'}$)", "red"),
        "bkg_tbc_matched": (["ttbarsl", "wwsl", "twsl"], "~d.is_wcb & d.is_tbc_matched", r"Bkg ($t_{bc}$)", "orange"),
        "bkg_tbq_matched": (["ttbarsl", "wwsl", "twsl"], "~d.is_wcb & d.is_tbq_matched", r"Bkg ($t_{bq'}$)", "wheat"),
        "bkg_non_matched": (["ttbarsl", "wwsl", "twsl"], "~d.is_wcb & d.is_non_matched", r"Bkg (non)", "lightyellow"),

        # signals
        "sig_t_matched": (["ttbarsl_wcb", "wwsl_wcb", "twsl_wcb"], "d.is_wcb & d.is_t_matched", r"Sig ($t_{bqq'}$)", "blue"),
        "sig_w_matched": (["ttbarsl_wcb", "wwsl_wcb", "twsl_wcb"], "d.is_wcb & d.is_w_matched", r"Sig ($W_{qq'}$)", "red"),
        "sig_tbc_matched": (["ttbarsl_wcb", "wwsl_wcb", "twsl_wcb"], "d.is_wcb & d.is_tbc_matched", r"Sig ($t_{bc}$)", "orange"),
        "sig_tbq_matched": (["ttbarsl_wcb", "wwsl_wcb", "twsl_wcb"], "d.is_wcb & d.is_tbq_matched", r"Sig ($t_{bq'}$)", "wheat"),
        "sig_non_matched": (["ttbarsl_wcb", "wwsl_wcb", "twsl_wcb"], "d.is_wcb & d.is_non_matched", r"Sig (non)", "lightyellow"),
    },

    variables = {
        # for template making
        "sophon_discr2_dnn_hist2d": (("d.fj_sophon_discr2[:,0]", "d.score_is_w_matched"), (bh.axis.Regular(100, 0.9, 1), bh.axis.Regular(100, 0, 1))),
        "sophon_discr3_dnn_hist2d": (("d.fj_sophon_discr3[:,0]", "d.score_is_w_matched"), (bh.axis.Regular(100, 0.9, 1), bh.axis.Regular(100, 0, 1))),
    },

    signal_mul_factor = 10,

    categories_merged = {
        "bkg_wjets": ["bkg_wjets"],
        "bkg_allwhad_tbc": ["bkg_tbc_matched"],
        "bkg_allwhad_others": ["bkg_t_matched", "bkg_w_matched", "bkg_tbq_matched", "bkg_non_matched"],
        "sig": ["sig_t_matched", "sig_w_matched", "sig_tbc_matched", "sig_tbq_matched", "sig_non_matched"],
    }
)
# btag_wp = {"L": 0.0557, "M": 0.297, "T": 0.725}


In [4]:
import pickle
with open("pickles/wcb_ana_templ_2dhist_spdiscr2_newsample_v0216.pkl", "rb") as f:
    templs_summary = pickle.load(f)

In [8]:
## write templates
filedir = './datacards/boosted'
aux_weight_scale = 3000. / 450 # for HL-LHC
# aux_weight_scale = 1.
aux_flavor_strength = 1

# 0.xxx, 0.xx
sophon_discr_thres = 0.95; evt_dnn_thres = 0.87  # 0.951, 0.87 optimal for strength=1; 0.95, 0.86 optimal for strength=0.5

########################################################
import copy
_templs = copy.deepcopy(templs_summary) ## important!

hist_out = {}
# should first sum over the first Sophon discr index (start:end:bh.sum)
for cm in config.categories_merged:
    hist_out[cm] = _templs["nom"][cm][int(sophon_discr_thres*1000)-900::bh.sum, :] ## important!

for target_flv in ["bjet", "cjet", "ljet"]:
    for target_region in ["B1", "B2", "C1", "C2", "N"]:
        for target_variation in ["up", "down"]:
            _variation = "Up" if target_variation == "up" else "Down"
            # get the inclusive non-flavour-cut template's total yield
            nevt_tot_nom = sum(_templs["nom"][cm].view(flow=True).value.sum() for cm in config.categories_merged)
            nevt_tot_flv = sum(_templs[f"{target_flv}_{target_region}_{target_variation}"][cm].view(flow=True).value.sum() for cm in config.categories_merged)
            fac =  nevt_tot_nom / nevt_tot_flv

            for cm in config.categories_merged:
                n = f"{cm}_ftag_{target_flv}_{target_region}{_variation}"
                hist_out[n] = _templs[f"{target_flv}_{target_region}_{target_variation}"][cm][int(sophon_discr_thres*1000)-900::bh.sum, :] ## important!
                # make sure the total yields doesn't change
                hist_out[n].view().value *= fac
                hist_out[n].view().variance *= fac**2
                # hist_out[n].view().value *= sum(hist_out[cm].view().value) / sum(hist_out[n].view().value)
                # hist_out[n].view().variance *= (sum(hist_out[cm].view().value) / sum(hist_out[n].view().value))**2

hist_out['data_obs'] = sum(hist_out[cm] for cm in config.categories_merged)

# aux weight scale
for n in hist_out:
    hist_out[n].view().value *= aux_weight_scale
    hist_out[n].view().variance *= aux_weight_scale**2


# rebin the DNN score
# === old setup ===
# for n in hist_out:
#     # hist_out[n] = hist_out[n][bh.loc(0.96):bh.loc(1.0):bh.rebin(4)] # 1 bin from 0.96--1
#     hist_out[n] = hist_out[n][bh.loc(0.6):bh.loc(1.0):bh.rebin(4)]
#     hist_out[n].view().variance *= 0
# ======
def rebin_hist_var_width(orig_hist, new_axis):
    new_hist = bh.Histogram(new_axis, storage=bh.storage.Weight())
    for i in range(orig_hist.axes[0].size):
        bin_center = orig_hist.axes[0].centers[i]
        if bin_center > new_axis.edges[0] and bin_center < new_axis.edges[-1]:
            new_idx = np.searchsorted(new_axis.edges, bin_center) - 1
            new_hist.view().value[new_idx] += orig_hist.view().value[i]
            new_hist.view().variance[new_idx] += orig_hist.view().variance[i]
    return new_hist

# write two templates: SR1 and SR2
hist_out1, hist_out2 = {}, {}
for n in hist_out:
    hist_out1[n] = rebin_hist_var_width(hist_out[n], bh.axis.Variable([evt_dnn_thres, 1.0])) # pass DNN bin
    hist_out1[n].view().value = np.maximum(hist_out1[n].view().value, 1e-3)
    hist_out1[n].view().variance *= 0

    hist_out2[n] = rebin_hist_var_width(hist_out[n], bh.axis.Variable([0, evt_dnn_thres])) # fail DNN bin
    hist_out2[n].view().value = np.maximum(hist_out2[n].view().value, 1e-3)
    hist_out2[n].view().variance *= 0

import re
for n in hist_out:
    if 'ftag' in n:
        hist_out1[n].view().value = hist_out1[n].view().value * aux_flavor_strength + hist_out1[re.sub(r"(_ftag_.*)", "", n)].view().value * (1 - aux_flavor_strength)
        hist_out2[n].view().value = hist_out2[n].view().value * aux_flavor_strength + hist_out2[re.sub(r"(_ftag_.*)", "", n)].view().value * (1 - aux_flavor_strength)

print('Apply aux weight scale:', aux_weight_scale)
print(hist_out1['data_obs'].values().sum(), hist_out2['data_obs'].values().sum())

with uproot.recreate(f'{filedir}/input_SR1.root') as fw:
    for n in hist_out1:
        fw[n] = hist_out1[n]
with uproot.recreate(f'{filedir}/input_SR2.root') as fw:
    for n in hist_out2:
        fw[n] = hist_out2[n]

lines = open(f'{filedir}/datacard.txt').readlines()
for i, line in enumerate(lines):
    if line.startswith('observation'):
        lines[i] = f'observation {hist_out1["data_obs"].values().sum()} {hist_out2["data_obs"].values().sum()}\n'
with open(f'{filedir}/datacard.txt', 'w') as f:
    f.writelines(lines)

Apply aux weight scale: 6.666666666666667
876.1882682047756 182198.26210505684


# Resolved

In [9]:
# Categories
from types import SimpleNamespace
config = SimpleNamespace(
    categories = { # category name: (sample list, selection, label, color)
        "bkg_wjets": (["wjets"], "d.lep_pt > 0", r"Bkg: W+jets", "lightcyan"),
        "bkg_ttbarsl": (["ttbarsl"], "~d.is_wcb", r"Bkg ($t\overline{t}$)", "orange"),
        "bkg_twsl": (["twsl"], "~d.is_wcb", r"Bkg ($tW$)", "magenta"),
        "bkg_wwsl": (["wwsl"], "~d.is_wcb", r"Bkg (WW)", "green"),

        "sig_ttbarsl": (["ttbarsl_wcb"], "d.is_wcb", r"Sig ($t\overline{t}$)", "orange"),
        "sig_twsl": (["twsl_wcb"], "d.is_wcb", r"Sig ($tW$)", "magenta"),
        "sig_wwsl": (["wwsl_wcb"], "d.is_wcb", r"Sig (WW)", "green"),
    },

    variables = {
        "evt_dnn_score_0p9_1_1000": ("d.evt_dnn_score", bh.axis.Regular(1000, 0.9, 1)), # to make templates
    },

    signal_mul_factor = 10,

    categories_merged = {
        "bkg_wjets": ["bkg_wjets"],
        "bkg_allwhad": ["bkg_ttbarsl", "bkg_twsl", "bkg_wwsl"],
        "sig": ["sig_ttbarsl", "sig_twsl", "sig_wwsl"],
    },
)
# btag_wp = {"L": 0.0557, "M": 0.297, "T": 0.725}


In [10]:
import pickle

# with open("pickles/wcb_rsvana_templ_newsample_v0216_nbc3.pkl", "rb") as f:
#     templs_summary = pickle.load(f)
# to exclude fatjet phase space from resolved channel, use this one
with open("pickles/wcb_rsvana_templ_newsample_v0216_nbc3_excludefj.pkl", "rb") as f:
    templs_summary = pickle.load(f)

with open("pickles/wcb_rsvana_templ_newsample_v0216.pkl", "rb") as f:
    templs_summary_orig = pickle.load(f)

In [ ]:
## new impl. for templs adding a flavour tagger cut

## write templates
filedir = './datacards/resolved'
aux_weight_scale = 3000. / 450 # for HL-LHC
aux_flavor_strength = 1. *1/np.sqrt(3000/140)
# aux_flavor_strength = 0.5

# 0.xxx, 0.xx
evt_dnn_thres = 0.990  # 0.990-0.992 all optimal..; for stregth=0.5, 0.986; for stregth=0.2, 0.983

########################################################
import copy
_templs = copy.deepcopy(templs_summary) ## important!
_templs0 = copy.deepcopy(templs_summary_orig) ## important!

hist_out = {}
for cm in config.categories_merged:
    hist_out[cm] = _templs["nom"][cm]

for target_flv in ["bjet", "cjet", "ljet"]:
    for target_region in ["B1", "B2", "C1", "C2", "N"]:
        for target_variation in ["up", "down"]:
            _variation = "Up" if target_variation == "up" else "Down"
            # get the inclusive non-flavour-cut template's total yield
            nevt_tot_nom = sum(_templs0["nom"][cm].view(flow=True).value.sum() for cm in config.categories_merged)
            nevt_tot_flv = sum(_templs0[f"{target_flv}_{target_region}_{target_variation}"][cm].view(flow=True).value.sum() for cm in config.categories_merged)
            fac =  nevt_tot_nom / nevt_tot_flv

            for cm in config.categories_merged:
                n = f"{cm}_ftag_{target_flv}_{target_region}{_variation}"
                hist_out[n] = _templs[f"{target_flv}_{target_region}_{target_variation}"][cm]
                # make sure the total yields doesn't change (use non-flavour-cut template)
                hist_out[n].view().value *= fac
                hist_out[n].view().variance *= fac**2

hist_out['data_obs'] = sum(hist_out[cm] for cm in config.categories_merged)

# aux weight scale
for n in hist_out:
    hist_out[n].view().value *= aux_weight_scale
    hist_out[n].view().variance *= aux_weight_scale**2

# rebin the DNN score
# === old setup ===
# for n in hist_out:
#     # hist_out[n] = hist_out[n][bh.loc(0.96):bh.loc(1.0):bh.rebin(4)] # 1 bin from 0.96--1
#     hist_out[n] = hist_out[n][bh.loc(0.6):bh.loc(1.0):bh.rebin(4)]
#     hist_out[n].view().variance *= 0
# ======
def rebin_hist_var_width(orig_hist, new_axis):
    new_hist = bh.Histogram(new_axis, storage=bh.storage.Weight())
    for i in range(orig_hist.axes[0].size):
        bin_center = orig_hist.axes[0].centers[i]
        if bin_center > new_axis.edges[0] and bin_center < new_axis.edges[-1]:
            new_idx = np.searchsorted(new_axis.edges, bin_center) - 1
            new_hist.view().value[new_idx] += orig_hist.view().value[i]
            new_hist.view().variance[new_idx] += orig_hist.view().variance[i]
    return new_hist

for n in hist_out:
    hist_out[n] = rebin_hist_var_width(hist_out[n], bh.axis.Variable([evt_dnn_thres, 1]))
    hist_out[n].view().value = np.maximum(hist_out[n].view().value, 1e-3)
    hist_out[n].view().variance *= 0

import re
for n in hist_out:
    if 'ftag' in n:
        hist_out[n].view().value = hist_out[n].view().value * aux_flavor_strength + hist_out[re.sub(r"(_ftag_.*)", "", n)].view().value * (1 - aux_flavor_strength)

print('Apply aux weight scale:', aux_weight_scale)
print(hist_out['data_obs'].values().sum())

with uproot.recreate(f'{filedir}/input_SR.root') as fw:
    for n in hist_out:
        fw[n] = hist_out[n]

lines = open(f'{filedir}/datacard.txt').readlines()
for i, line in enumerate(lines):
    if line.startswith('observation'):
        lines[i] = f'observation {hist_out["data_obs"].values().sum()}\n'
with open(f'{filedir}/datacard.txt', 'w') as f:
    f.writelines(lines)

Apply aux weight scale: 6.666666666666667
348.50599602581934
